In [ ]:
! pip install -r requirements.txt

In [1]:
import os
import json
import requests
import time

from openai import AzureOpenAI
from pathlib import Path
from typing import Optional
from IPython.display import clear_output

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_KEY")

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

bing_search_subscription_key = os.getenv("BING_SEARCH_SUBSCRIPTION_KEY")
bing_search_url = f"{os.getenv('BING_SEARCH_URL')}v7.0/search"

In [3]:
def bing_search(query: str) -> list:
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_search_subscription_key}
    params = {
        "q": query,
        "textDecorations": False,
        # "customconfig": "2711e09d-ea3b-4847-b7f4-d1e6a234745f"
    }
    
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})
    # print(output)
    return json.dumps(output)


# bing_search("who is the ceo of Goldman Sachs")

In [4]:

client = AzureOpenAI(api_key=aoai_api_key, api_version=api_version, azure_endpoint=azure_endpoint)

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

available_functions = {
    "search_bing": bing_search,
}

def get_tool_response(messages):
    
    current_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

    response_message = current_response.choices[0].message
    tool_calls = response_message.tool_calls
    messages.append(response_message)

    is_calling_function = True if tool_calls else False
    print(f"calling function: {is_calling_function}")

    while is_calling_function:

        tool_call = tool_calls[0]

        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(**function_args)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
        current_response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        response_message = current_response.choices[0].message
        # print(response_message)
        messages.append(response_message)
        tool_calls = response_message.tool_calls
        is_calling_function = True if tool_calls else False
    
    # print("messages:")
    # print(messages[-1].content)

    return messages[-1].content


In [5]:
instructions = """
You are an assistant designed to help people answer questions.
You have access to query the web using Bing Search.
You should call bing search whenever a question requires up to date information or could benefit from web data.
Always include the Citation when looking at bing search with URL.
Answer in readable Markdown format.
"""
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "What is the Max Request (tokens) and Training Data (up to) for Azure OpenAI gpt-4-32k(0314)?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
The Azure OpenAI GT-4-32k (0314) model has the following specifications:

- **Max Request Tokens**: 32,768 tokens
- **Training Data**: Up to September 2021

For more detailed information, you can refer to the official documentation on [Azure OpenAI Service Models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models).


In [14]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "how are you doing?"},
]

response = get_tool_response(messages)
print(response)

calling function: False
I'm doing great! How can I assist you today?


In [8]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "Who is the CEO of Goldman Sachs?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
The current CEO of Goldman Sachs is David Solomon. He has held the position since October 2018. For more details, you can visit his [profile on Goldman Sachs' website](https://www.goldmansachs.com/our-firm/our-people-and-leadership/leadership/executive-officers/david-solomon) or check [Wikipedia](https://en.wikipedia.org/wiki/David_M._Solomon) for additional information.


In [11]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "Who is the COO of Goldman Sachs?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
The current Chief Operating Officer (COO) of Goldman Sachs is John Waldron. You can find more details about him and the leadership team on the [Goldman Sachs Leadership](https://www.goldmansachs.com/our-firm/our-people-and-leadership/leadership) page.


In [12]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "What is Goldman Sachs Net worth?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
As of the most recent data available, Goldman Sachs has a net worth (market capitalization) of approximately $165.83 billion as of October 2024. This figure represents the company's market cap, which is calculated by multiplying the current stock price by the number of shares outstanding.

For more detailed and recent financial information, you can refer to their latest reports and press releases available on the [Goldman Sachs website](https://www.goldmansachs.com/investor-relations/financials/current/annual-reports/2023-annual-report/).

[Citation](https://stockanalysis.com/stocks/gs/market-cap/)


In [23]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "What are some of the AI initive within Goldman Sachs?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
Goldman Sachs is actively engaged in several artificial intelligence initiatives. Here are some key highlights:

1. **AI Forward 2023**: Goldman Sachs, in partnership with SV Angel, hosted AI Forward 2023 in San Francisco. The event brought together nearly 400 leaders and investors across the tech ecosystem to discuss current AI advancements. The event was headlined by Bill Gates and featured leading experts from various fields to debate the dimensions and future of AI [Source](https://www.goldmansachs.com/what-we-do/investment-banking/insights/articles/ai-forward-2023).

2. **Office of Applied Innovation**: Directed by George Lee and Jared Cohen, this office aims to provide insights at the intersection of geopolitics and technology, playing a crucial role in understanding and leveraging AI for strategic benefits [Source](https://www.forbes.com/sites/peterhigh/2023/07/14/goldman-sachs-co-heads-of-applied-innovation-gen-ai-is-the-worlds-greatest-experiment-in-anar